Global setup


In [2]:
n=4
k=2
q=41
eta=2
R.<x> = ZZ[]

Bob's setup

In [3]:
Ax = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
Ax  #public

[ 27*x^3 + 4*x^2 + 32*x + 33  23*x^3 + 5*x^2 + 11*x + 18]
[ 11*x^3 + 24*x^2 + 33*x + 8 28*x^3 + 16*x^2 + 26*x + 18]

In [4]:
def binomial_centered():
    a1=randrange(0,2)
    a2=randrange(0,2)
    b1=randrange(0,2)
    b2=randrange(0,2)
#    return a1+a2-b1-b2
    return a1-b1
    return 0

In [5]:
sx = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
sx  #private

[-x^3 - x^2 + 1]
[    -x^3 - x^2]

In [6]:
ex = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
ex #private

[         x^3]
[-x^3 - x - 1]

In [7]:
tx = matrix(((Ax*sx + ex) % (x^n+1) % q))
tx  #public

[  16*x^3 + 3*x^2 + 9*x + 3]
[7*x^3 + 37*x^2 + 29*x + 24]

**Matrices for lattice attack**


In [27]:
matrix.options.max_rows(49) 

In [8]:
def negacirculant(v):
    Nshift = matrix([[0, 0, 0, -1],             #Negacyclic shift matrix
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0]])
    return sum([v[i]*Nshift^i for i in range(0, len(v))])

In [9]:
Ax

[ 27*x^3 + 4*x^2 + 32*x + 33  23*x^3 + 5*x^2 + 11*x + 18]
[ 11*x^3 + 24*x^2 + 33*x + 8 28*x^3 + 16*x^2 + 26*x + 18]

In [10]:
A = block_matrix([[negacirculant(Ax[0,0].padded_list(4)), negacirculant(Ax[0,1].padded_list(4))], 
                  [negacirculant(Ax[1,0].padded_list(4)), negacirculant(Ax[1,1].padded_list(4))]])
A

[ 33 -27  -4 -32| 18 -23  -5 -11]
[ 32  33 -27  -4| 11  18 -23  -5]
[  4  32  33 -27|  5  11  18 -23]
[ 27   4  32  33| 23   5  11  18]
[---------------+---------------]
[  8 -11 -24 -33| 18 -28 -16 -26]
[ 33   8 -11 -24| 26  18 -28 -16]
[ 24  33   8 -11| 16  26  18 -28]
[ 11  24  33   8| 28  16  26  18]

In [11]:
tx

[  16*x^3 + 3*x^2 + 9*x + 3]
[7*x^3 + 37*x^2 + 29*x + 24]

In [12]:
t = block_matrix([[column_matrix(1, 4, tx[0,0].padded_list(4))],
                [column_matrix(1, 4, tx[1,0].padded_list(4))]])
t

[ 3]
[ 9]
[ 3]
[16]
[--]
[24]
[29]
[37]
[ 7]

In [13]:
I8 = identity_matrix(8)
I8

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]

In [29]:
M = block_matrix([[A.transpose()], [I8], [-t.transpose()], [q*I8]])
M

[ 33  32   4  27   8  33  24  11]
[-27  33  32   4 -11   8  33  24]
[ -4 -27  33  32 -24 -11   8  33]
[-32  -4 -27  33 -33 -24 -11   8]
[ 18  11   5  23  18  26  16  28]
[-23  18  11   5 -28  18  26  16]
[ -5 -23  18  11 -16 -28  18  26]
[-11  -5 -23  18 -26 -16 -28  18]
[-------------------------------]
[  1   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0]
[  0   0   1   0   0   0   0   0]
[  0   0   0   1   0   0   0   0]
[  0   0   0   0   1   0   0   0]
[  0   0   0   0   0   1   0   0]
[  0   0   0   0   0   0   1   0]
[  0   0   0   0   0   0   0   1]
[-------------------------------]
[ -3  -9  -3 -16 -24 -29 -37  -7]
[-------------------------------]
[ 41   0   0   0   0   0   0   0]
[  0  41   0   0   0   0   0   0]
[  0   0  41   0   0   0   0   0]
[  0   0   0  41   0   0   0   0]
[  0   0   0   0  41   0   0   0]
[  0   0   0   0   0  41   0   0]
[  0   0   0   0   0   0  41   0]
[  0   0   0   0   0   0   0  41]

**Idea for lattice attack**


$$A(x) \star \mathbf{s}(x) + \mathbf{e}(x) - \mathbf{t}(x) \equiv \mathbf{0} \pmod{q}$$



so $$A \mathbf{s} + \mathbf{e} - \mathbf{t} \equiv \mathbf{0} \pmod{q}$$ for suitable matrices/vectors


or in block row form, $$(\mathbf{s}^T | \mathbf{e}^T | 1 | \mathbf{s'}^T)\begin{pmatrix} A^T \\ - \\ I \\ - \\ -\mathbf{t}^T \\ - \\ qI \end{pmatrix} =  \mathbf{s''}M = \mathbf{0} $$

so $\mathbf{s''}$ is a short vector in the kernel of $M$.

In [30]:
sT = block_matrix([[matrix(1, 4, sx[0,0].padded_list(4)), matrix(1, 4, sx[1,0].padded_list(4))]])
sT

[ 1  0 -1 -1| 0  0 -1 -1]

In [31]:
eT = block_matrix([[matrix(1, 4, ex[0,0].padded_list(4)), matrix(1, 4, ex[1,0].padded_list(4))]])
eT

[ 0  0  0  1|-1 -1  0 -1]

In [34]:
spp = block_matrix([[s, e, matrix([1]), matrix([[0,0,0,0,0,0,0,0]])]])
spp

[ 1  0 -1 -1  0  0 -1 -1| 0  0  0  1 -1 -1  0 -1| 1| 0  0  0  0  0  0  0  0]

In [35]:
spp*M

[ 82  82   0 -82  82  82   0 -82]

In [38]:
spp = block_matrix([[s, e, matrix([1]), matrix([[-2,-2,0,2,-2,-2,0,2]])]])
spp

[ 1  0 -1 -1  0  0 -1 -1| 0  0  0  1 -1 -1  0 -1| 1|-2 -2  0  2 -2 -2  0  2]

In [39]:
spp*M

[0 0 0 0 0 0 0 0]

In [45]:
M.transpose().right_kernel_matrix(basis='LLL')  #There no left_kernel_matrix command for some reason

[ 1  0 -1 -1  0  0 -1 -1  0  0  0  1 -1 -1  0 -1  1 -2 -2  0  2 -2 -2  0  2]
[-3 -1 -1  0  2  0 -1  2 -1 -1 -1 -2 -1  2  3  3 -3  1  1  3  0 -2 -1  1  0]
[ 1 -2 -1  1 -1  2 -1  0  0  4 -2  4 -4 -1  2  2  0  0 -1  3  0  1 -1  1  2]
[-1 -1 -4  2  2  1  1 -1  0  2  2 -1 -3  4 -1  3  2  1 -1  4  2  0  1  2  2]
[ 0  1  1  0 -1 -2 -1  1  3  5 -3 -1  0 -5 -1 -3 -2  0  0  0 -1 -1  0  0  0]
[ 5  1 -1  0 -3 -3  2  2 -2  5 -2  1  1 -5  1 -2  0 -3 -2  1 -2  0  1  0  0]
[-1  1 -2 -1  0 -1  2 -1 -4  0  4 -2 -2 -4 -1  2  0  0  0 -1  3 -2  1 -1  1]
[ 0  1 -1 -3  0  1  2  4 -2 -1  0 -4  4 -6  0  2 -2  0 -1 -1  0  0 -1 -2 -3]
[ 0 -2  0  0  5  2  0 -3  4 -4  1  1  3  0 -2 -1  4 -3  0 -1  0 -1 -2  0 -1]
[-2 -1  1 -1  2 -1  0  1  4 -2  4  0 -1  2  2  4  0 -1  3  0  0 -1  1  2 -1]
[ 1  1  1  1  1 -4 -1  1  5  0 -6  1 -2 -1  2  2 -1 -2  0  1 -3 -2  0  1 -1]
[ 2  1  0  0  2 -1  1  0  1 -1 -4 -5  2 -5  2 -2  4 -2 -1 -1 -1  1  1  1 -2]
[ 0  1 -2  1 -1  0  0 -3 -2  4  0 -3  1  1  4  0  1  1 -2  0  3 -1  0 -1  3]

**Eve's attack**

Eve knows $A$ so she can set up $M$:

In [46]:
M

[ 33  32   4  27   8  33  24  11]
[-27  33  32   4 -11   8  33  24]
[ -4 -27  33  32 -24 -11   8  33]
[-32  -4 -27  33 -33 -24 -11   8]
[ 18  11   5  23  18  26  16  28]
[-23  18  11   5 -28  18  26  16]
[ -5 -23  18  11 -16 -28  18  26]
[-11  -5 -23  18 -26 -16 -28  18]
[-------------------------------]
[  1   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0]
[  0   0   1   0   0   0   0   0]
[  0   0   0   1   0   0   0   0]
[  0   0   0   0   1   0   0   0]
[  0   0   0   0   0   1   0   0]
[  0   0   0   0   0   0   1   0]
[  0   0   0   0   0   0   0   1]
[-------------------------------]
[ -3  -9  -3 -16 -24 -29 -37  -7]
[-------------------------------]
[ 41   0   0   0   0   0   0   0]
[  0  41   0   0   0   0   0   0]
[  0   0  41   0   0   0   0   0]
[  0   0   0  41   0   0   0   0]
[  0   0   0   0  41   0   0   0]
[  0   0   0   0   0  41   0   0]
[  0   0   0   0   0   0  41   0]
[  0   0   0   0   0   0   0  41]

In [76]:
H, U = M.hermite_form(transformation = true)
U  #The transformation matrix that puts M into a (not very interesting) Hermite Normal Form

[   -14304971208804184798668      4847233526041633497876      2055629147630155029924      9950175238776752510412     10260392826451837147032       990313628143577281068     -9421391472116633655516     -9144218001448806044724    619964266519688077300021                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0]
[    -9047975507289166658685      3065902725812047122531      1300197106889678576475      6293542331513235242127      6489756717005111502567       626378992392728647950     -5959083596884798610211     -5783769803010866068305    392130918474847246153140                          16                  

In [51]:
N = U[8:25]
N  #This should give a basis of the left kernel

[                -5379339747                  1822787029                   773012921                  3741732323                  3858388678                   372404347                 -3542884839                 -3438654621                233135626205                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0                           0]
[    -3015991835763055552895      1021967575270682374177       433399035629892858825      2097847443837745080709      2163252239001703834189       208792997464242882650     -1986361198961599536737     -1927923267670288689435    130710306158282415384380                           5                  

In [52]:
N.LLL()

[ 1  0 -1 -1  0  0 -1 -1  0  0  0  1 -1 -1  0 -1  1 -2 -2  0  2 -2 -2  0  2]
[-3 -1 -1  0  2  0 -1  2 -1 -1 -1 -2 -1  2  3  3 -3  1  1  3  0 -2 -1  1  0]
[ 2  1 -1  1 -2  1  0 -1 -4  2 -4  0  1 -2 -2 -4  0  1 -3  0  0  1 -1 -2  1]
[ 0  1  1  0 -1 -2 -1  1  3  5 -3 -1  0 -5 -1 -3 -2  0  0  0 -1 -1  0  0  0]
[-5  0  2  0  2  1 -3 -1  5  0 -1 -2 -1  0 -2 -1 -2  3  2 -1  1 -1 -1  0  0]
[ 2 -1 -1  1  2 -1  3 -1  0 -3  5  0 -2  1  2  2 -1 -3  0  0 -3 -2 -1 -3 -2]
[ 1  2 -2  2  0 -2  3 -1 -6 -3 -3 -1 -2 -1  0  4  0  1 -1  0 -1  0  2 -2 -1]
[ 0  0 -1  0  0 -2  1  0 -1  0 -1  6  1  0 -3  5  1 -1  1  1  1 -1  2  2  1]
[ 0 -1  1 -2  0  2  0  0 -6 -2 -1 -4 -4  0  0  2 -2 -1  0 -2  0 -1 -3 -3 -1]
[ 0 -1  2 -1  1  0  0  3  2 -4  0  3 -1 -1 -4  0 -1 -1  2  0 -3  1  0  1 -3]
[ 2  2  1 -2 -5 -3  0  1  1  3  4  2 -4 -1  1  1  0 -1  0 -2  2  0  2  1  2]
[ 0  2  2 -3 -4  3 -2  3  1 -1 -2 -1  2  0 -1 -1 -2  3 -2 -3  1  3 -2 -2 -1]
[ 1 -1  2  1  2 -4  1 -2 -1 -1  0  2  1 -4  2 -2  0 -4  3 -1 -3 -3  1  0 -1]

**Eve's attack showing calculation of transformation matrix**


In [57]:
M

[ 33  32   4  27   8  33  24  11]
[-27  33  32   4 -11   8  33  24]
[ -4 -27  33  32 -24 -11   8  33]
[-32  -4 -27  33 -33 -24 -11   8]
[ 18  11   5  23  18  26  16  28]
[-23  18  11   5 -28  18  26  16]
[ -5 -23  18  11 -16 -28  18  26]
[-11  -5 -23  18 -26 -16 -28  18]
[-------------------------------]
[  1   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0]
[  0   0   1   0   0   0   0   0]
[  0   0   0   1   0   0   0   0]
[  0   0   0   0   1   0   0   0]
[  0   0   0   0   0   1   0   0]
[  0   0   0   0   0   0   1   0]
[  0   0   0   0   0   0   0   1]
[-------------------------------]
[ -3  -9  -3 -16 -24 -29 -37  -7]
[-------------------------------]
[ 41   0   0   0   0   0   0   0]
[  0  41   0   0   0   0   0   0]
[  0   0  41   0   0   0   0   0]
[  0   0   0  41   0   0   0   0]
[  0   0   0   0  41   0   0   0]
[  0   0   0   0   0  41   0   0]
[  0   0   0   0   0   0  41   0]
[  0   0   0   0   0   0   0  41]

In [56]:
M1 = block_matrix([[M, identity_matrix(25)]])
M1

[ 33  32   4  27   8  33  24  11|  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[-27  33  32   4 -11   8  33  24|  0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ -4 -27  33  32 -24 -11   8  33|  0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[-32  -4 -27  33 -33 -24 -11   8|  0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ 18  11   5  23  18  26  16  28|  0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[-23  18  11   5 -28  18  26  16|  0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ -5 -23  18  11 -16 -28  18  26|  0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[-11  -5 -23  18 -26 -16 -28  18|  0   0   0   0   0   0   0  

In [63]:
H1 = M1.hermite_form()
H1 # The right side should be a transformation matrix (after some extra unnecessary but not harmful reduction)

[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 38 40 22 33  4 15 25 27  2  5  1 10 15 19 24  4]
[ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 26 29 26 10 40 14 25 11  0  1  0  3  6  6  9  1]
[ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  9 23 16  0 18  3 36 32  3  6  1 12 19 

In [72]:
N1 = H1.submatrix(8, 8)
N1 # A basis for the kernel

[ 1  0  0  0  0  0  0  0  0 26 29 26 10 40 14 25 11  0  1  0  3  6  6  9  1]
[ 0  1  0  0  0  0  0  0  0  9 23 16  0 18  3 36 32  3  6  1 12 19 22 28  4]
[ 0  0  1  0  0  0  0  0  0 15  4 15 33 27 11 26 26  2  6  1  9 15 18 23  3]
[ 0  0  0  1  0  0  0  0  0 31 36 15 23 29 40 13  3  1  0  0  0  2  2  2  0]
[ 0  0  0  0  1  0  0  0  0  2 13 32  3 25  1 14  6  0  1  0  1  3  3  5  0]
[ 0  0  0  0  0  1  0  0  0 36  7  9  8 33 32 26  6  1  0  0  2  4  3  4  0]
[ 0  0  0  0  0  0  1  0  0  8 18 17 17  7 16 17 12  1  3  0  4  7  9 10  1]
[ 0  0  0  0  0  0  0  1  0 13 12 19 20 19 29 11 10  1  2  1  3  6  7  9  1]
[ 0  0  0  0  0  0  0  0  1  3  1 19  8 37 26 16 14  1  3  1  5  8  9 12  2]
[ 0  0  0  0  0  0  0  0  0 41  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0 41  0  0  0  0  0  0  0  0 -1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0 41  0  0  0  0  0  0  0  0 -1  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0 41  0  0  0  0  0  0  0  0 -1  0  0  0]

In [73]:
N1.LLL()

[ 1  0 -1 -1  0  0 -1 -1  0  0  0  1 -1 -1  0 -1  1 -2 -2  0  2 -2 -2  0  2]
[-3 -1 -1  0  2  0 -1  2 -1 -1 -1 -2 -1  2  3  3 -3  1  1  3  0 -2 -1  1  0]
[ 2  1 -1  1 -2  1  0 -1 -4  2 -4  0  1 -2 -2 -4  0  1 -3  0  0  1 -1 -2  1]
[-2  1  1 -1 -2  1 -3  1  0  3 -5  0  2 -1 -2 -2  1  3  0  0  3  2  1  3  2]
[-1  0  0  2  2  1  0 -3  1 -1  0 -2 -2 -1 -2 -3 -3  1 -1 -1 -2 -2 -3 -5 -1]
[ 1  2 -2  2  0 -2  3 -1 -6 -3 -3 -1 -2 -1  0  4  0  1 -1  0 -1  0  2 -2 -1]
[ 1  0  1 -2 -2  1 -1  3  0  1  1 -4  1  1  5 -2  2  0  0  0  1  3  1  3  0]
[ 0  1 -1  2  0 -2  0  0  6  2  1  4  4  0  0 -2  2  1  0  2  0  1  3  3  1]
[ 0  1 -2  1 -1  0  0 -3 -2  4  0 -3  1  1  4  0  1  1 -2  0  3 -1  0 -1  3]
[ 4  1  1  1  0 -2  1 -2  3 -4 -2  4 -1  2  0  2  4 -3 -1 -2 -2  1  1  0 -1]
[ 0  1  1  0 -1 -2 -1  1  3  5 -3 -1  0 -5 -1 -3 -2  0  0  0 -1 -1  0  0  0]
[-5  0 -1  1  0  2 -3 -1 -1  0 -2  4  0 -4  2  0 -3  5  0  2  3 -1 -1  0  3]
[ 1  1  4 -2 -2 -1 -1  1  0 -2 -2  1  3 -4  1 -3 -2 -1  1 -4 -2  0 -1 -2 -2]

In [74]:
s

[ 1  0 -1 -1| 0  0 -1 -1]

The secret key is recovered.